In [1]:
import machine_umls_annotation
import pandas as pd

from google.cloud import spanner
from google.cloud.spanner_admin_database_v1.types import spanner_database_admin
import os
import json

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/dgg32/.config/gcloud/application_default_credentials.json"

In [3]:
if not os.path.exists("temp/trials_umls.tsv"):

    query_column = "Medicine or Vaccine (generic name)"
    df = machine_umls_annotation.machine_assign_umls("neo4j/trials.tsv", query_column)
    df[(df['CUI_umls'].str.strip() != "") & pd.notnull(df['CUI_umls'])].to_csv("temp/trials_umls.tsv", sep='\t', index=False)
else:

    df = pd.read_csv("temp/trials_umls.tsv", sep='\t', lineterminator='\n')

#df[pd.notnull(df['CUI_umls'])].to_csv("temp/trials_umls.tsv", sep='\t', index=False)

In [4]:
df.head()

,Posting ID,Sponsor,Study Title,Medicine or Vaccine (generic name),Sponsor Identification Number,Medical Condition,Phase,Link to Sponsor Study Registry,Link to study details on ClinicalTrials.gov (if available),Link to study details on EudraCT (if available),...,Dataset specifications,Protocol with any amendments,Raw dataset,Reporting and analysis plan,Additional Information about the data and documents available for this study,Date Added to this Site,CUI_umls,preferred_umls_name,semantic_type,high_confidence
0,1,GSK,"U0289-401: An Evaluator Blinded, 8 Week, Split...",benzoyl peroxide/salicylic acid,STF114550,Acne Vulgaris,Phase 4,https://www.gsk-studyregister.com/en/trial-det...,http://clinicaltrials.gov/show/NCT01706250,NaN,...,Y,N,Y,N,An analysis-ready dataset and a reporting and ...,1-Aug-2015,C0005088;C0036079,benzoyl peroxide;salicylic acid,Organic Chemical|Pharmacologic Substance;Organ...,True;True
1,3,GSK,"GSK1550188, A randomised, single-blind, placeb...",belimumab,BEL114243,Systemic Lupus Erythematosus,Phase 1,https://www.gsk-studyregister.com/en/trial-det...,http://clinicaltrials.gov/show/NCT01381536,NaN,...,Y,N,Y,Y,An analysis-ready dataset is not available for...,1-Jul-2015,C1723401,belimumab,"Amino Acid, Peptide, or Protein|Pharmacologic ...",True
2,4,GSK,"A Multicenter, Randomized, Double-Blind, Phase...","calcipotriol, calcipotriene",STF114741,Psoriasis,Phase 3,https://www.gsk-studyregister.com/en/trial-det...,http://clinicaltrials.gov/show/NCT00688519,NaN,...,Y,Y,Y,Y,NaN,1-Jul-2015,C0065767,calcipotriene,Organic Chemical|Pharmacologic Substance,True
3,8,GSK,Two-week Study to Determine and Compare the To...,benzoyl peroxide/clindamycin phosphate,STF114546,Acne Vulgaris,Phase 4,https://www.gsk-studyregister.com/en/trial-det...,http://clinicaltrials.gov/show/NCT01015638,NaN,...,Y,Y,Y,N,An analysis-ready dataset and reporting and an...,1-Aug-2014,C0005088;C0055881,benzoyl peroxide;clindamycin phosphate,Organic Chemical|Pharmacologic Substance;Organ...,True;True
4,9,GSK,"A single-blind, randomized, comparative split-...",benzoyl peroxide/clindamycin phosphate,STF114547,Acne Vulgaris,Phase 4,https://www.gsk-studyregister.com/en/trial-det...,http://clinicaltrials.gov/show/NCT00964223,NaN,...,Y,Y,N,Y,A raw dataset is not available for this study.,1-Aug-2014,C0005088;C0055881,benzoyl peroxide;clindamycin phosphate,Organic Chemical|Pharmacologic Substance;Organ...,True;True


In [5]:
spanner_client = spanner.Client()

project_id = "spanner-432202"
project_location = "asia-northeast1"
instance_id = "drug-disease-trials"
database_id = "drug-disease-trial-db"

# Get a Cloud Spanner instance by ID.
#instance = spanner_client.instance("drug-disease-trials")
#database = instance.database("drug-disease-trial")


In [6]:

database_admin_api = spanner_client.database_admin_api

request = spanner_database_admin.CreateDatabaseRequest(
        parent=database_admin_api.instance_path(spanner_client.project, instance_id),
        create_statement=f"CREATE DATABASE `{database_id}`",
        extra_statements=[
            """CREATE TABLE Trials (
            PostingID     INT64 NOT NULL,
            Sponsor    STRING(1024),
            StudyTitle     STRING(1024),
            Drug   STRING(1024),
            Disorder   STRING(1024),
            Phase   STRING(1024),
            LinkToSponsorStudyRegistry   STRING(1024),
            LinkToClinicalTrials   STRING(1024),
            cui  ARRAY<STRING(MAX)>,
            PreferredUMLSName   ARRAY<STRING(MAX)>,
            StudyTitle_Tokens TOKENLIST AS (TOKENIZE_FULLTEXT(StudyTitle)) HIDDEN,
            ) PRIMARY KEY (PostingID)""",

            """CREATE SEARCH INDEX StudyIndex
            ON Trials(StudyTitle_Tokens)
            """,

            """CREATE TABLE Drug (
            drug_cui  STRING(1024) NOT NULL,
            name  STRING(1024)
            ) PRIMARY KEY (drug_cui)""",

            """CREATE TABLE Disorder (
            disorder_cui  STRING(1024) NOT NULL,
            name  STRING(1024),
            definition  STRING(3072),
            definitionEmbedding ARRAY<FLOAT32>(vector_length=>768)
            ) PRIMARY KEY (disorder_cui)""",

            """CREATE TABLE DrugDisorder (
            drug_cui  STRING(1024) NOT NULL,
            disorder_cui  STRING(1024) NOT NULL,
            FOREIGN KEY (disorder_cui) REFERENCES Disorder (disorder_cui)
            ) PRIMARY KEY (drug_cui, disorder_cui),
            INTERLEAVE IN PARENT Drug ON DELETE CASCADE""",

            """CREATE TABLE MOA (
            moa_id  STRING(1024),
            name  STRING(1024)
            ) PRIMARY KEY (moa_id)""",

            """CREATE TABLE DrugMOA (
            drug_cui  STRING(1024) NOT NULL,
            moa_id  STRING(1024) NOT NULL,
            FOREIGN KEY (moa_id) REFERENCES MOA (moa_id)
            ) PRIMARY KEY (drug_cui, moa_id),
            INTERLEAVE IN PARENT Drug ON DELETE CASCADE""",

            """CREATE OR REPLACE PROPERTY GRAPH DrugGraph
            NODE TABLES (Drug, Disorder, MOA)
            EDGE TABLES (
                DrugDisorder
                    SOURCE KEY (drug_cui) REFERENCES Drug (drug_cui)
                    DESTINATION KEY (disorder_cui) REFERENCES Disorder (disorder_cui)
                    LABEL MAY_TREAT,
                DrugMOA
                    SOURCE KEY (drug_cui) REFERENCES Drug (drug_cui)
                    DESTINATION KEY (moa_id) REFERENCES MOA (moa_id)
                    LABEL HAS_MECHANISM_OF_ACTION
            )
            """,

            f"""CREATE MODEL EmbeddingsModel INPUT(
            content STRING(MAX),
                ) OUTPUT(
                embeddings STRUCT<statistics STRUCT<truncated BOOL, token_count FLOAT32>, values ARRAY<FLOAT32>>,
            ) REMOTE OPTIONS (
                endpoint = '//aiplatform.googleapis.com/projects/{project_id}/locations/{project_location}/publishers/google/models/text-embedding-004'
                )
            """,

            f"""CREATE MODEL LLMModel INPUT(
                prompt STRING(MAX),
                ) OUTPUT(
                content STRING(MAX),
                ) REMOTE OPTIONS (
                endpoint = '//aiplatform.googleapis.com/projects/{project_id}/locations/{project_location}/publishers/google/models/gemini-pro',
                default_batch_size = 1
                )
            """


        ]
    )

operation = database_admin_api.create_database(request=request)



I0000 00:00:1724378668.928429 7937224 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [7]:
df = pd.read_csv("temp/trials_umls.tsv", sep='\t', lineterminator='\n')
values = []

for index, row in df.iterrows():
    values.append((row["Posting ID"], row["Sponsor"], row["Study Title"], row["Medicine or Vaccine (generic name)"], row["Medical Condition"], row["Phase"], row["Link to Sponsor Study Registry"], row["Link to study details on ClinicalTrials.gov (if available)"], row["CUI_umls"].split(";"), row["preferred_umls_name"].split(";")))

print (values[:2])

[(1, 'GSK', 'U0289-401: An Evaluator Blinded, 8 Week, Split-Face Study to Evaluate and Compare the Efficacy and Tolerability of MAXCLARITY II and PROACTIV in Subjects with Acne', 'benzoyl peroxide/salicylic acid', 'Acne Vulgaris', 'Phase 4', 'https://www.gsk-studyregister.com/en/trial-details/?id=114550', 'http://clinicaltrials.gov/show/NCT01706250', ['C0005088', 'C0036079'], ['benzoyl peroxide', 'salicylic acid']), (3, 'GSK', 'GSK1550188, A randomised, single-blind, placebo controlled, dose ascending, single dose study to evaluate the safety, tolerability, pharmacokinetics, and pharmacodynamic of GSK1550188 in Japanese subjects with Systemic Lupus Erythematosus (SLE)', 'belimumab', 'Systemic Lupus Erythematosus', 'Phase 1', 'https://www.gsk-studyregister.com/en/trial-details/?id=114243', 'http://clinicaltrials.gov/show/NCT01381536', ['C1723401'], ['belimumab'])]


In [8]:
instance = spanner_client.instance(instance_id)
database = instance.database(database_id)

with database.batch() as batch:
        batch.insert(
            table="Trials",
            columns=("PostingID", "Sponsor", "StudyTitle", "Drug", "Disorder", "Phase", "LinkToSponsorStudyRegistry", "LinkToClinicalTrials", "cui", "PreferredUMLSName"),
            values=values
        )

I0000 00:00:1724378679.042221 7937224 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


In [9]:

values = []

for line in open("spanner/drugs_4.json", "r"):
    value = json.loads(line)

    values.append((value["cui"], value["name"]))

print (values[:2])

[('C0048318', 'mequinol'), ('C0035629', 'rimantadine')]


In [10]:
with database.batch() as batch:
        batch.insert(
            table="Drug",
            columns=("drug_cui", "name"),
            values=values
        )

In [11]:
values = []

for line in open("spanner/disease_4.json", "r"):
    value = json.loads(line)

    values.append((value["cui"], value["name"], value["definition"]))

print (values[:2])

[('C0023321', 'Lentigo', "Small circumscribed melanoses resembling, but differing histologically from, freckles. The concept includes senile lentigo ('liver spots') and nevoid lentigo (nevus spilus, lentigo simplex) and may also occur in association with multiple congenital defects or congenital syndromes (e.g., Peutz-Jeghers syndrome)."), ('C0021400', 'Influenza', 'An acute viral infection in humans involving the respiratory tract. It is marked by inflammation of the NASAL MUCOSA; the PHARYNX; and conjunctiva, and by headache and severe, often generalized, myalgia.')]


In [12]:
with database.batch() as batch:
        batch.insert(
            table="Disorder",
            columns=("disorder_cui", "name", "definition"),
            values=values
        )

In [13]:
values = []

df = pd.read_csv("spanner/moa_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    values.append((row["id"], row["name"]))

print (values[:2])


[('N0000175850', 'Melanin Synthesis Inhibitors'), ('N0000175542', 'M2 Protein Inhibitors')]


In [14]:
with database.batch() as batch:
        batch.insert(
            table="MOA",
            columns=("moa_id", "name"),
            values=values
        )

In [15]:
values = []

df = pd.read_csv("spanner/cui_indication_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    values.append((row["from"], row["to"]))

print (values[:2])

[('C0048318', 'C0023321'), ('C0035629', 'C0021400')]


In [16]:
with database.batch() as batch:
        batch.insert(
            table="DrugDisorder",
            columns=("drug_cui", "disorder_cui"),
            values=values
        )

In [17]:
values = []

df = pd.read_csv("spanner/cui_moa_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
    values.append((row["from"], row["to"]))

print (values[:2])

[('C0046056', 'N0000000205'), ('C4550366', 'N0000187062')]


In [18]:
with database.batch() as batch:
        batch.insert(
            table="DrugMOA",
            columns=("drug_cui", "moa_id"),
            values=values
        )

In [21]:

def create_embeddings(transaction):
    row_ct = transaction.execute_update(
        """
        UPDATE Disorder
        SET
        Disorder.definitionEmbedding = (
            SELECT embeddings.values
            FROM SAFE.ML.PREDICT(
            MODEL EmbeddingsModel,
            (SELECT Disorder.definition AS content)
            ) @{remote_udf_max_rows_per_rpc=200}
        )
        WHERE Disorder.definitionEmbedding IS NULL AND Disorder.definition <> ''
        """
    )


database.run_in_transaction(create_embeddings)

In [20]:
### This does not work for granular instances (instance with just one node)

# operation = database.update_ddl([
#     """
#         CREATE VECTOR INDEX ConditionsDefinitionEmbeddingIndex
#             ON Conditions(definitionEmbedding)
#             WHERE definitionEmbedding IS NOT NULL
#         OPTIONS (distance_type = 'COSINE')
# """])
    
# print('Waiting for operation to complete...')
# operation.result()